## Preparing Data

In [1]:


%matplotlib inline
import numpy as np 
import pandas as pd 
import torch
import torchtext
from torchtext import data
import spacy
import os
import re


os.environ['OMP_NUM_THREADS'] = '4'


SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(lower=True,include_lengths=True ,tokenize='spacy')

LABEL = data.Field(sequential=False, 
                         use_vocab=False, 
                         pad_token=None, 
                            unk_token=None, dtype = torch.float)




dataFields = {"comment_text": ("comment_text", TEXT), 
              'toxic': ("toxic", LABEL), 
              'severe_toxic': ("severe_toxic", LABEL),
              'threat': ("threat", LABEL), 
              'obscene': ("obscene", LABEL),
              'insult': ("insult", LABEL), 
              'identity_hate': ("identity_hate", LABEL)}

dataset= data.TabularDataset(path='./data/train.json', 
                                            format='json',
                                            fields=dataFields, 
                                            skip_header=True)

In [186]:
import random
SEED = 3
train, unimportant = dataset.split(split_ratio=0.5,random_state = random.seed(SEED)) 

train_data, val_data = dataset.split(split_ratio=0.5,random_state = random.seed(SEED))

In [187]:
MAX_VOCAB_SIZE = 20_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "fasttext.simple.300d", 
                 unk_init = torch.Tensor.normal_)

In [188]:
BATCH_SIZE = 512

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, val_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.comment_text),
    sort_within_batch = True,
    device = device)

In [189]:
yFields = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
iaux=0
for batch in valid_iterator:
    iaux+=1
    aux = batch
    aux2= torch.stack([getattr(batch, y) for y in yFields])
    if iaux==20: break

In [190]:

        
torch.transpose( torch.stack([getattr(aux, y) for y in yFields]),0,1)

tensor([[0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [191]:
aux.comment_text[0].size()

torch.Size([12, 512])

In [192]:
aux.toxic.size()

torch.Size([512])

## Build model

In [193]:
import torch.nn as nn
from torch.functional import F

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, dense_dim ,output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc1 = nn.Linear(hidden_dim * 2, dense_dim)
        
        self.fc2 = nn.Linear(dense_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
        
        #Doc missing below
        dense = self.fc1(hidden)
        
        dense = F.relu(dense)
        
        return self.fc2(dense)

In [194]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 30 #256
DENSE_DIM = 30
OUTPUT_DIM = 6 #1 
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM,
            DENSE_DIM,
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [195]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,104,376 trainable parameters


In [196]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([20002, 300])


In [197]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-1.3904, -1.1406, -1.3349,  ..., -1.4462, -0.5370, -0.0938],
        [-0.0913, -0.1363,  0.8198,  ...,  0.9718, -0.2894,  0.4969],
        [-0.1609,  0.2147,  0.6759,  ...,  0.2905, -0.3324, -0.5295],
        ...,
        [-1.2045,  0.4061, -1.0270,  ...,  1.6240,  0.3288,  0.1658],
        [-0.9369, -0.9798, -1.0953,  ..., -0.6119, -1.1121,  0.0215],
        [ 0.2444, -0.0492,  0.4232,  ..., -0.0952, -0.0125,  0.2102]])

In [198]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1609,  0.2147,  0.6759,  ...,  0.2905, -0.3324, -0.5295],
        ...,
        [-1.2045,  0.4061, -1.0270,  ...,  1.6240,  0.3288,  0.1658],
        [-0.9369, -0.9798, -1.0953,  ..., -0.6119, -1.1121,  0.0215],
        [ 0.2444, -0.0492,  0.4232,  ..., -0.0952, -0.0125,  0.2102]])


## Train our model

In [199]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [200]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [201]:
import numpy
from sklearn.metrics import roc_auc_score
def roc_auc(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    #rounded_preds = torch.sigmoid(preds)
    
    #assert preds.size()==y.size()
    
    #reds=rounded_preds.detach().numpy()

    #y=y.numpy()
    
    global var_y
    global var_preds
    var_y = y
    var_preds = preds
    print('jeje', y.shape)
    acc = roc_auc_score(y, preds)
    print('jojo',preds.shape)
    
    return acc

In [202]:


def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    preds_list=[]
    labels_list= []
 
    
    for i, batch in enumerate(iterator):
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.comment_text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        batch_labels=torch.stack([getattr(batch, y) for y in yFields]) #transpose?
        batch_labels = torch.transpose(batch_labels,0,1)
        
        loss = criterion(predictions, batch_labels)
        
        loss.backward()
        
        optimizer.step()
        
        preds_list+=[torch.sigmoid(predictions).detach().numpy()]
        labels_list+=[batch_labels.numpy()]
        
        #if i%64==0:
        #    epoch_acc += [roc_auc(np.vstack(preds_list), np.vstack(batch_labels))]
        #    preds_list=[]
        #    labels_list= []
            
        
        epoch_loss += loss.item()
        
        
        
    return epoch_loss / len(iterator), roc_auc(np.vstack(preds_list), np.vstack(labels_list))

In [203]:


def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    preds_list=[]
    labels_list= []
    epoch_acc=[]
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.comment_text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            batch_labels = torch.stack([getattr(batch, y) for y in yFields]) #transpose?
            batch_labels = torch.transpose(batch_labels,0,1)
            
            loss = criterion(predictions, batch_labels)

            epoch_loss += loss.item()
            
            preds_list+=[torch.sigmoid(predictions).detach().numpy()]
            labels_list+=[batch_labels.numpy()]
        
            #if i%64==0:
            #    epoch_acc += [roc_auc(np.vstack(preds_list), np.vstack(batch_labels))]
            #    preds_list=[]
            #    labels_list= []
        
    return epoch_loss / len(iterator), roc_auc(np.vstack(preds_list), np.vstack(labels_list))



In [204]:
from torchsummary import summary

In [205]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [206]:


N_EPOCHS = 8

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    print('jaja')
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    print('juju')
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')



jeje (79785, 6)
jojo (79785, 6)
jaja
jeje (79785, 6)
jojo (79785, 6)
juju
Epoch: 01 | Epoch Time: 1m 22s
	Train Loss: 0.235 | Train Acc: 50.97%
	 Val. Loss: 0.139 |  Val. Acc: 72.93%
jeje (79785, 6)
jojo (79785, 6)
jaja
jeje (79785, 6)
jojo (79785, 6)
juju
Epoch: 02 | Epoch Time: 1m 20s
	Train Loss: 0.119 | Train Acc: 80.17%
	 Val. Loss: 0.085 |  Val. Acc: 93.48%
jeje (79785, 6)
jojo (79785, 6)
jaja
jeje (79785, 6)
jojo (79785, 6)
juju
Epoch: 03 | Epoch Time: 1m 19s
	Train Loss: 0.077 | Train Acc: 93.08%
	 Val. Loss: 0.068 |  Val. Acc: 94.88%
jeje (79785, 6)
jojo (79785, 6)
jaja
jeje (79785, 6)
jojo (79785, 6)
juju
Epoch: 04 | Epoch Time: 1m 22s
	Train Loss: 0.067 | Train Acc: 94.71%
	 Val. Loss: 0.067 |  Val. Acc: 94.95%
jeje (79785, 6)
jojo (79785, 6)
jaja
jeje (79785, 6)
jojo (79785, 6)
juju
Epoch: 05 | Epoch Time: 1m 20s
	Train Loss: 0.062 | Train Acc: 95.47%
	 Val. Loss: 0.065 |  Val. Acc: 95.19%
jeje (79785, 6)
jojo (79785, 6)
jaja
jeje (79785, 6)
jojo (79785, 6)
juju
Epoch: 06 |

In [ ]:
var_preds.shape

In [173]:
var_y

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        ...,
        [1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])